In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import copy

In [2]:
link_df = pd.read_csv('astrohud/assets/data/constellation_links.csv')
nodes = set(list(link_df.Item1.unique()) + list(link_df.Item2.unique()))

In [3]:
g = nx.Graph()
g.add_nodes_from(nodes)
for _, row in link_df.iterrows():
    g.add_edge(row.Item1, row.Item2)

In [49]:
#g = nx.minors.contracted_nodes(g, 'serpens_cauda', 'serpens_caput', self_loops=False)
#g = nx.relabel_nodes(g, {'serpens_cauda': 'serpens'})
#g.remove_edge('hydra', 'libra')

In [44]:
class Coloring:
    def __init__(self, max_color=4):
        self.colors = tuple()
        self.max_color = max_color

    def set_order(self, node_order):
        self.node_order = node_order
        self.node_indices = {node: node_order.index(node) for node in node_order}

    def is_done(self):
        return len(self.colors) >= len(self.node_order)

    def make_child(self, color):
        child = copy.copy(self)
        child.colors = self.colors + (color,)
        return child
    
    def get_children(self, g):
        assert not self.is_done()
        next_node = self.node_order[len(self.colors)]
        allowed = set(range(self.max_color))
        for neighbor in g.adj[next_node]:
            i = self.node_indices[neighbor]
            if i >= len(self.colors):
                continue
            allowed.discard(self.colors[i])

        return [self.make_child(c) for c in allowed]

    def __hash__(self):
        return hash(self.colors)

    def __eq__(self, other):
        return self.colors == other.colors

    def __repr__(self):
        return repr(self.colors)
        

In [74]:
#node_order = list(nx.coloring.strategy_connected_sequential_bfs(g, []))

special = dict(
    aries=0,
    taurus=1,
    gemini=2,
    cancer=3,
    leo=0,
    virgo=1,
    libra=2,
    scorpio=3,
    sagittarius=0,
    capricorn=1,
    aquarius=2,
    pisces=3,
    hydra=2,
    cetus=2,
    serpens_cauda=2,
    serpens_caput=3,
    aquila=3,
    corona_borealis=1,
)

node_order = list(special.keys())
queue = list(node_order)

while len(node_order) < len(g.nodes):
    node = queue.pop(0)
    for child in g.adj[node]:
        if child in node_order:
            continue
        node_order.append(child)
        if child in queue:
            continue
        queue.append(child)

In [72]:
base_coloring = Coloring(4)
base_coloring.set_order(node_order)

for i in range(len(special)):
    c = special[node_order[i]]
    base_coloring = base_coloring.make_child(c)

base_coloring

(0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 2, 2, 2, 3)

In [73]:
colorings = [base_coloring]
history = set()

found = []
while len(colorings) > 0:
    item = colorings.pop(-1)
    if item.is_done():
        found.append(item)
        continue

    for child in item.get_children(g):
        if child in history:
            continue
        history.add(child)
        colorings.append(child)

    if len(history) % 1000 == 0:
        print(len(history), len(found), end='\r')

print(len(history), len(found))

696000 222

KeyboardInterrupt: 